# Making Pandas DataFrames from API Requests
In this example, we will use the U.S. Geological Survey's API to grab a JSON object of earthquake data and convert it to a `pandas.DataFrame`.

USGS API: https://earthquake.usgs.gov/fdsnws/event/1/

### Get Data from API

In [2]:
import datetime as dt
import pandas as pd
import requests

yesterday = dt.date.today() - dt.timedelta(days=1)
api = 'https://earthquake.usgs.gov/fdsnws/event/1/query'
payload = {
    'format': 'geojson',
    'starttime': yesterday - dt.timedelta(days=30),
    'endtime': yesterday
}
response = requests.get(api, params=payload)

# let's make sure the request was OK
response.status_code

200

Response of 200 means OK, so we can pull the data out of the result. Since we asked the API for a JSON payload, we can extract it from the response with the `json()` method.

### Isolate the Data from the JSON Response
We need to check the structures of the response data to know where our data is.

In [6]:
earthquake_json = response.json()
earthquake_json.keys()

dict_keys(['type', 'metadata', 'features', 'bbox'])

The USGS API provides information about our request in the `metadata` key. Note that your result will be different, regardless of the date range you chose, because the API includes a timestamp for when the data was pulled:

In [7]:
earthquake_json['metadata']

{'generated': 1692459612000,
 'url': 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson&starttime=2023-07-19&endtime=2023-08-18',
 'title': 'USGS Earthquakes',
 'status': 200,
 'api': '1.14.0',
 'count': 10900}

Each element in the JSON array `features` is a row of data for our dataframe.

In [8]:
type(earthquake_json['features'])

list

Your data will be different depending on the date you run this.

In [9]:
earthquake_json['features'][0]

{'type': 'Feature',
 'properties': {'mag': 1.2,
  'place': '23 km N of Fishhook, Alaska',
  'time': 1692316189219,
  'updated': 1692316300547,
  'tz': None,
  'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/ak023aj421vt',
  'detail': 'https://earthquake.usgs.gov/fdsnws/event/1/query?eventid=ak023aj421vt&format=geojson',
  'felt': None,
  'cdi': None,
  'mmi': None,
  'alert': None,
  'status': 'automatic',
  'tsunami': 0,
  'sig': 22,
  'net': 'ak',
  'code': '023aj421vt',
  'ids': ',ak023aj421vt,',
  'sources': ',ak,',
  'types': ',origin,phase-data,',
  'nst': None,
  'dmin': None,
  'rms': 0.47,
  'gap': None,
  'magType': 'ml',
  'type': 'earthquake',
  'title': 'M 1.2 - 23 km N of Fishhook, Alaska'},
 'geometry': {'type': 'Point', 'coordinates': [-149.2266, 61.9588, 15.8]},
 'id': 'ak023aj421vt'}

### Convert to DataFrame
We need to grab the `properties` section out of every entry in the `features` JSON array to create our dataframe.

In [11]:
earthquake_properties_data = [
    quake['properties'] for quake in earthquake_json['features']
]
df = pd.DataFrame(earthquake_properties_data)
df.head()

,mag,place,time,updated,tz,url,detail,felt,cdi,mmi,...,ids,sources,types,nst,dmin,rms,gap,magType,type,title
0,1.20,"23 km N of Fishhook, Alaska",1692316189219,1692316300547,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",ak023aj421vt,",",ak,",",origin,phase-data,",NaN,NaN,0.47,NaN,ml,earthquake,"M 1.2 - 23 km N of Fishhook, Alaska"
1,0.66,"65 km NNE of Chignik, Alaska",1692316051020,1692401263310,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av92006736,",",av,",",origin,phase-data,",4.0,NaN,0.20,266.0,ml,earthquake,"M 0.7 - 65 km NNE of Chignik, Alaska"
2,0.19,None,1692315806180,1692400669570,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av91113713,",",av,",",origin,phase-data,",5.0,NaN,0.02,145.0,ml,earthquake,M 0.2 -
3,0.31,Alaska Peninsula,1692315788170,1692400546700,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",av92006731,",",av,",",origin,phase-data,",8.0,NaN,0.07,91.0,ml,earthquake,M 0.3 - Alaska Peninsula
4,0.35,"3 km W of Cobb, CA",1692315397500,1692318975371,None,https://earthquake.usgs.gov/earthquakes/eventp...,https://earthquake.usgs.gov/fdsnws/event/1/que...,NaN,NaN,NaN,...,",nc73925381,",",nc,",",nearby-cities,origin,phase-data,scitech-link,",18.0,0.0104,0.02,124.0,md,earthquake,"M 0.4 - 3 km W of Cobb, CA"


### (Optional) Write Data to CSV

In [7]:
df.to_csv('earthquakes.csv', index=False)

<hr>
<div>
    <a href="./2-creating_dataframes.ipynb">
        <button style="float: left;">&#8592; Previous Notebook</button>
    </a>
    <a href="./4-inspecting_dataframes.ipynb">
        <button style="float: right;">Next Notebook &#8594;</button>
    </a>
</div>
<br>
<hr>